# Попытки улучшить уже существующую модель


## Для начала посмотрим на модель ruBERT и проверим качество zero-shot предсказаний

In [1]:
import numpy as np

print(np.__version__)

1.26.4


In [2]:
!pip uninstall -y transformers accelerate
!pip install torch==2.1.2
!pip install transformers==4.38.2
!pip install accelerate==0.28.0
!pip install datasets==2.18.0
!pip install scikit-learn
!pip install "numpy<2"

Found existing installation: transformers 4.38.2
Uninstalling transformers-4.38.2:
  Successfully uninstalled transformers-4.38.2
Found existing installation: accelerate 0.28.0
Uninstalling accelerate-0.28.0:
  Successfully uninstalled accelerate-0.28.0



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\gfgh\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


  Using cached transformers-4.38.2-py3-none-any.whl.metadata (130 kB)
Using cached transformers-4.38.2-py3-none-any.whl (8.5 MB)



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\gfgh\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


  Using cached accelerate-0.28.0-py3-none-any.whl.metadata (18 kB)
Using cached accelerate-0.28.0-py3-none-any.whl (290 kB)



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\gfgh\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\gfgh\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\gfgh\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\gfgh\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


- первым делом подключим датасет `lamoda`

In [3]:
import pandas as pd
import torch
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import classification_report

df = pd.read_csv("lamoda_reviews.csv").dropna(subset=["Review", "Rating"])
df["label"] = df["Rating"] - 1 
print(df["label"].value_counts())

label
4    99278
3     7315
2     3352
0     1425
1     1094
Name: count, dtype: int64


- сделаем балансировку выборки, пускай будет по 1000 отзывов каждого вида

In [4]:
df_balanced = df.groupby("label").apply(lambda x: x.sample(min(1000, len(x)))).reset_index(drop=True)
texts = df_balanced["Review"].tolist()
labels = df_balanced["label"].tolist()
print(df_balanced["label"].value_counts())

label
0    1000
1    1000
2    1000
3    1000
4    1000
Name: count, dtype: int64


C:\Users\gfgh\AppData\Local\Temp\ipykernel_240\2951978859.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_balanced = df.groupby("label").apply(lambda x: x.sample(min(1000, len(x)))).reset_index(drop=True)


- пускай теперь наша модель сделает предсказания без обучения на этих данных

In [5]:
model_name = "DeepPavlov/rubert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5)
model.eval()

all_preds = []
with torch.no_grad():
    for i in range(0, len(texts), 8):
        batch = texts[i:i+8]
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=128)
        outputs = model(**inputs)
        preds = outputs.logits.argmax(-1).tolist()
        all_preds.extend(preds)

C:\Users\gfgh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\gfgh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and 

- пора вывести результаты и метрики

In [6]:
unique_labels = sorted(set(labels + all_preds))
target_names = [f"{i+1}★" for i in unique_labels]

print("\nZERO-SHOT КАЧЕСТВО RuBERT")
print(classification_report(labels, all_preds, labels=unique_labels, target_names=target_names, digits=3))


ZERO-SHOT КАЧЕСТВО RuBERT
              precision    recall  f1-score   support

          1★      0.231     0.027     0.048      1000
          2★      0.000     0.000     0.000      1000
          3★      0.198     0.216     0.207      1000
          4★      0.000     0.000     0.000      1000
          5★      0.202     0.765     0.319      1000

    accuracy                          0.202      5000
   macro avg      0.126     0.202     0.115      5000
weighted avg      0.126     0.202     0.115      5000



C:\Users\gfgh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\gfgh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\gfgh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.

In [7]:
from collections import Counter

pred_counts = Counter(all_preds)
print("Частота предсказаний по классам:")
for i in range(5):
    print(f"{i+1}★: {pred_counts.get(i, 0)}")

Частота предсказаний по классам:
1★: 117
2★: 0
3★: 1089
4★: 0
5★: 3794


### Вывод

Модель в zero-shot режиме демонстрирует крайне слабые результаты и серьезный перекос предсказаний в пользу одного класса. Использование модели без дообучения на размеченных отзывах Lamoda неприемлемо. Необходимо провести fine-tuning модели на собственных данных, чтобы добиться адекватного качества предсказаний.

# Fine-tuning

Загружаем датасет Lamoda, оставляем только допустимые оценки от 1 до 5, удаляет пропущенные отзывы и создаёт целевую метку label, сдвинутую от 0 до 4 (для обучения модели). Затем переименовывает колонку Review в text, чтобы HuggingFace Dataset распознал вход.

In [8]:
import pandas as pd
import numpy as np
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import (AutoTokenizer, AutoModelForSequenceClassification,
                          TrainingArguments, Trainer)
from sklearn.metrics import accuracy_score, f1_score, classification_report
df = pd.read_csv("lamoda_reviews.csv")
df = df[df["Rating"].isin([1, 2, 3, 4, 5])] 
df = df.dropna(subset=["Review"])
df["label"] = df["Rating"] - 1  
df = df[["Review", "label"]].rename(columns={"Review": "text"})
df= df.groupby("label").apply(lambda x: x.sample(min(1000, len(x)))).reset_index(drop=True)
print(len(df))


5000


C:\Users\gfgh\AppData\Local\Temp\ipykernel_240\2746065884.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df= df.groupby("label").apply(lambda x: x.sample(min(1000, len(x)))).reset_index(drop=True)


Выбираем 80% записей для обучения и 20% — для оценки, при этом сохраняем пропорции классов.

In [9]:
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df["label"], random_state=42)

Создаём специализированные датасеты для обучения модели через библиотеку HuggingFace.

In [10]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

Здесь мы применяем токенизацию: каждое текстовое поле превращается в набор токенов, пригодных для подачи в BERT.

In [11]:
model_name = "DeepPavlov/rubert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

C:\Users\gfgh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Модель RuBERT получает классификационную голову с пятью выходами — по числу звёзд.

In [12]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Мы хотим видеть точность и взвешенный F1-score, чтобы оценить общее качество и учитывать баланс классов.

In [13]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1_weighted": f1_score(labels, predictions, average="weighted")
    }

Параметры задают стратегию обучения, частоту сохранения, число эпох, размер батча и логгирование. Выбираем лучшую модель по F1.

In [14]:
import transformers
print(transformers.__version__)
from transformers.training_args import TrainingArguments

4.38.2


In [15]:
print(TrainingArguments.__module__)

transformers.training_args


In [16]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)

Теперь всё готово для обучения: мы объединяем модель, данные, метрики и параметры в Trainer и запускаем процесс.

In [17]:
import transformers
import accelerate
import datasets
import torch

print("Transformers:", transformers.__version__)
print("Accelerate:", accelerate.__version__)
print("Datasets:", datasets.__version__)
print("Torch:", torch.__version__)

Transformers: 4.38.2
Accelerate: 0.28.0
Datasets: 2.18.0
Torch: 2.1.2+cpu


- ограничим потребление иначе мой ноутбук домохозяйки не доживет до рассвета

In [18]:
from transformers import TrainingArguments


training_args = TrainingArguments(
    disable_tqdm=False,
    output_dir="./results",
    num_train_epochs=2,            
    per_device_train_batch_size=4,      
    per_device_eval_batch_size=4,     
    logging_strategy="epoch",                
    warmup_steps=0,
    evaluation_strategy="no", 
    save_strategy="no",            
    load_best_model_at_end=False,           
    report_to="none",
    seed=42
)


In [19]:
print(len(train_dataset))
print(len(test_dataset))

4000
1000


In [20]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()


C:\Users\gfgh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


  0%|          | 0/2000 [00:00<?, ?it/s]

{'loss': 1.4156, 'grad_norm': 8.860630989074707, 'learning_rate': 2.5e-05, 'epoch': 1.0}
{'loss': 1.1527, 'grad_norm': 15.66103458404541, 'learning_rate': 0.0, 'epoch': 2.0}
{'train_runtime': 3802.2015, 'train_samples_per_second': 2.104, 'train_steps_per_second': 0.526, 'train_loss': 1.2841529541015626, 'epoch': 2.0}


TrainOutput(global_step=2000, training_loss=1.2841529541015626, metrics={'train_runtime': 3802.2015, 'train_samples_per_second': 2.104, 'train_steps_per_second': 0.526, 'train_loss': 1.2841529541015626, 'epoch': 2.0})

После обучения проверяем качество модели на отложенной выборке и выводим точность, полноту и F1-score по каждому классу.

In [21]:
import numpy as np 

preds_output = trainer.predict(test_dataset)
preds = np.argmax(preds_output.predictions, axis=-1)
true = preds_output.label_ids

print(classification_report(true, preds, digits=3, target_names=[f"{i+1}★" for i in range(5)]))

  0%|          | 0/250 [00:00<?, ?it/s]

              precision    recall  f1-score   support

          1★      0.522     0.465     0.492       200
          2★      0.354     0.280     0.313       200
          3★      0.299     0.415     0.347       200
          4★      0.415     0.440     0.427       200
          5★      0.764     0.665     0.711       200

    accuracy                          0.453      1000
   macro avg      0.471     0.453     0.458      1000
weighted avg      0.471     0.453     0.458      1000



Ура! Выводы можно посмотреть в файлике report